In [1]:
import pandas as pd
import requests
from ipynb.fs.full.edgar_functions import *
from ipynb.fs.full.cik import headers

In [4]:
def get_facts(ticker, headers=headers):
    cik = cik_matching_ticker(ticker)
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    company_facts = requests.get(url, headers=headers).json()
    return company_facts

In [13]:
facts = get_facts("WSM")
facts['facts']['us-gaap']['AccountsPayableCurrent']['units']['USD']

[{'end': '2009-08-02',
  'val': 129798000,
  'accn': '0001193125-10-208171',
  'fy': 2010,
  'fp': 'Q2',
  'form': '10-Q',
  'filed': '2010-09-10',
  'frame': 'CY2009Q2I'},
 {'end': '2009-11-01',
  'val': 171904000,
  'accn': '0001193125-10-278135',
  'fy': 2010,
  'fp': 'Q3',
  'form': '10-Q',
  'filed': '2010-12-10',
  'frame': 'CY2009Q3I'},
 {'end': '2010-01-31',
  'val': 188241000,
  'accn': '0001193125-10-208171',
  'fy': 2010,
  'fp': 'Q2',
  'form': '10-Q',
  'filed': '2010-09-10'},
 {'end': '2010-01-31',
  'val': 188241000,
  'accn': '0001193125-10-278135',
  'fy': 2010,
  'fp': 'Q3',
  'form': '10-Q',
  'filed': '2010-12-10'},
 {'end': '2010-01-31',
  'val': 188241000,
  'accn': '0001193125-11-085046',
  'fy': 2010,
  'fp': 'FY',
  'form': '10-K',
  'filed': '2011-03-31',
  'frame': 'CY2009Q4I'},
 {'end': '2010-05-02',
  'val': 177938000,
  'accn': '0001193125-11-162733',
  'fy': 2011,
  'fp': 'Q1',
  'form': '10-Q',
  'filed': '2011-06-10',
  'frame': 'CY2010Q1I'},
 {'end': '

In [23]:
def facts_DF(ticker, headers=headers):
    facts = get_facts(ticker, headers)
    us_gaap_data = facts["facts"]["us-gaap"]
    df_data = []
    for fact, details in us_gaap_data.items():
        for unit in details["units"]:
            for item in details["units"][unit]:
                row = item.copy()
                row["fact"] = fact
                df_data.append(row)

    df = pd.DataFrame(df_data)
    df["end"] = pd.to_datetime(df["end"])
    df["start"] = pd.to_datetime(df["start"])
    df = df.drop_duplicates(subset=["fact", "end", "val"])
    df.set_index("end", inplace=True)
    labels_dict = {fact: details["label"] for fact, details in us_gaap_data.items()}
    return df, labels_dict

In [24]:
facts, label_dict=facts_DF('WSM')

In [25]:
facts_DF('WSM')

(                    val                  accn    fy  fp  form       filed  \
 end                                                                         
 2009-08-02  129798000.0  0001193125-10-208171  2010  Q2  10-Q  2010-09-10   
 2009-11-01  171904000.0  0001193125-10-278135  2010  Q3  10-Q  2010-12-10   
 2010-01-31  188241000.0  0001193125-10-208171  2010  Q2  10-Q  2010-09-10   
 2010-05-02  177938000.0  0001193125-11-162733  2011  Q1  10-Q  2011-06-10   
 2010-08-01  184135000.0  0001193125-10-208171  2010  Q2  10-Q  2010-09-10   
 ...                 ...                   ...   ...  ..   ...         ...   
 2023-10-29    2500000.0  0001628280-23-040136  2023  Q3  10-Q  2023-11-28   
 2024-01-28    2500000.0  0001628280-24-012221  2023  FY  10-K  2024-03-20   
 2022-01-30   11300000.0  0001628280-24-012221  2023  FY  10-K  2024-03-20   
 2023-01-29   11400000.0  0001628280-24-012221  2023  FY  10-K  2024-03-20   
 2024-01-28   13600000.0  0001628280-24-012221  2023  FY  10-K  